In [3]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	# 各种层类型的实现
import torch.nn.functional as F	# 各中层函数的实现，与层类型对应，如：卷积函数、池化函数、归一化函数等等
import torch.optim as optim	# 实现各种优化算法的包
import numpy as np

import argparse
import sys


import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
from grid_data import GridData

from compute_mpe import CircuitMPE
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')
import gurobipy
cmpe = CircuitMPE('/root/PySDD/notebooks/yahoo.vtree', '/root/PySDD/notebooks/yahoo.sdd')
class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=24
        self.card=20
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
#legalList={0: [46], 1: [59], 2: [53, 63, 86], 3: [17, 22, 23, 24, 26, 34, 51, 61, 66, 74, 81, 85, 87, 89, 90, 96], 4: [24, 26, 37, 71, 83, 87, 94, 99], 5: [96], 6: [38], 7: [85], 8: [19, 64], 9: [], 10: [], 11: [], 12: [17, 79, 81], 13: [17, 23, 24, 25, 26, 30, 36, 42, 54, 56, 61, 66, 68, 72, 77, 81, 84, 89, 92, 95], 14: [], 15: [], 16: [17, 24, 25, 27, 35, 39, 42, 46, 48, 56, 61, 66, 72, 76, 81, 84, 89], 17: [3, 12, 13, 16, 24, 25, 27, 29, 35, 39, 42, 46, 51, 52, 54, 56, 61, 66, 72, 76, 77, 79, 81, 85, 89, 92, 96], 18: [], 19: [8, 37, 64, 70, 91], 20: [], 21: [], 22: [3, 30, 61, 66, 78, 81, 85, 90, 96], 23: [3, 13, 61, 66, 89], 24: [3, 4, 13, 16, 17, 25, 26, 27, 31, 34, 38, 42, 51, 54, 56, 61, 66, 68, 72, 74, 76, 77, 81, 84, 87, 89, 99], 25: [13, 16, 17, 24, 27, 35, 36, 39, 40, 42, 48, 51, 52, 54, 56, 61, 66, 68, 72, 76, 77, 81, 84, 89, 92], 26: [3, 4, 13, 24, 34, 42, 54, 66, 74, 77, 81, 83, 92, 97], 27: [16, 17, 24, 25, 35, 39, 46, 51, 52, 61, 68, 76, 87, 99], 28: [], 29: [17, 35, 56, 66, 72, 79, 81, 89], 30: [13, 22, 54, 61, 66, 73, 75, 78, 81, 95], 31: [24, 46, 51, 52, 61, 76, 79], 32: [], 33: [], 34: [3, 24, 26, 54, 66, 71, 74, 77, 81, 84, 94], 35: [16, 17, 25, 27, 29, 36, 39, 40, 46, 48, 52, 56, 61, 66, 72, 76, 77, 81, 89], 36: [13, 25, 35, 42, 56, 57, 66, 77, 81, 84, 89, 92], 37: [4, 19, 51, 64, 87, 99], 38: [6, 24, 51, 63, 67, 99], 39: [16, 17, 25, 27, 35, 48, 56, 61, 76, 77, 81, 85, 89], 40: [25, 35], 41: [], 42: [13, 16, 17, 24, 25, 26, 36, 54, 56, 61, 66, 68, 72, 76, 77, 81, 84, 89, 92], 43: [], 44: [51, 74], 45: [], 46: [0, 16, 17, 27, 31, 35, 51, 52, 61, 76, 96], 47: [], 48: [16, 25, 35, 39, 56, 61, 66, 77, 89], 49: [], 50: [], 51: [3, 17, 24, 25, 27, 31, 37, 38, 44, 46, 61, 68, 76, 87, 94, 99], 52: [17, 25, 27, 31, 35, 46, 61, 72, 76, 79, 81, 89], 53: [2, 70, 86, 93], 54: [13, 17, 24, 25, 26, 30, 34, 42, 56, 61, 66, 68, 72, 77, 81, 84, 89, 92, 95], 55: [], 56: [13, 16, 17, 24, 25, 29, 35, 36, 39, 42, 48, 54, 61, 66, 77, 81, 84, 89, 92], 57: [36], 58: [], 59: [1], 60: [], 61: [3, 13, 16, 17, 22, 23, 24, 25, 27, 30, 31, 35, 39, 42, 46, 48, 51, 52, 54, 56, 66, 72, 76, 77, 79, 81, 84, 85, 87, 89, 90, 96], 62: [], 63: [2, 38], 64: [8, 19, 37, 74, 87, 91], 65: [], 66: [3, 13, 16, 17, 22, 23, 24, 25, 26, 29, 30, 34, 35, 36, 42, 48, 54, 56, 61, 72, 73, 74, 76, 77, 81, 84, 85, 89, 92], 67: [38], 68: [13, 24, 25, 27, 42, 51, 54, 87, 99], 69: [], 70: [19, 53], 71: [4, 34, 74], 72: [13, 16, 17, 24, 25, 29, 35, 42, 52, 54, 61, 66, 76, 79, 81, 89], 73: [30, 66, 78, 84], 74: [3, 24, 26, 34, 44, 64, 66, 71, 77, 83], 75: [30], 76: [16, 17, 24, 25, 27, 31, 35, 39, 42, 46, 51, 52, 61, 66, 72, 79, 81, 89, 96], 77: [13, 17, 24, 25, 26, 34, 35, 36, 39, 42, 48, 54, 56, 61, 66, 74, 81, 84, 89, 92], 78: [22, 30, 73, 85, 90], 79: [12, 17, 29, 31, 52, 61, 72, 76, 81, 89], 80: [], 81: [3, 12, 13, 16, 17, 22, 24, 25, 26, 29, 30, 34, 35, 36, 39, 42, 52, 54, 56, 61, 66, 72, 76, 77, 79, 84, 85, 89, 92, 95], 82: [], 83: [4, 26, 74], 84: [13, 16, 24, 25, 34, 36, 42, 54, 56, 61, 66, 73, 77, 81, 89], 85: [3, 7, 17, 22, 39, 61, 66, 78, 81, 89, 90], 86: [2, 53, 88, 91], 87: [3, 4, 24, 27, 37, 51, 61, 64, 68, 91, 94, 99], 88: [86], 89: [3, 13, 16, 17, 23, 24, 25, 29, 35, 36, 39, 42, 48, 52, 54, 56, 61, 66, 72, 76, 77, 79, 81, 84, 85, 92], 90: [3, 22, 61, 78, 85], 91: [19, 64, 86, 87, 94], 92: [13, 17, 25, 26, 36, 42, 54, 56, 66, 77, 81, 89, 95], 93: [53], 94: [4, 34, 51, 87, 91, 99], 95: [13, 30, 54, 81, 92], 96: [3, 5, 17, 22, 46, 61, 76], 97: [26], 98: [], 99: [4, 24, 27, 37, 38, 51, 68, 87, 94]}
legalList={0: [13, 18],
 1: [17, 20, 22],
 2: [4, 5, 10, 13, 14, 16, 17, 18, 20, 22],
 3: [5, 9, 11, 12, 15],
 4: [2, 10, 13, 14, 16, 17, 18, 20, 22],
 5: [2, 3, 6, 9, 10, 11, 12, 14, 16],
 6: [5, 15],
 7: [],
 8: [],
 9: [3, 5, 11, 12, 15],
 10: [2, 4, 5, 14, 16, 22],
 11: [3, 5, 9, 12, 15],
 12: [3, 5, 9, 11, 15],
 13: [0, 2, 4, 14, 18, 22],
 14: [2, 4, 5, 10, 13, 16, 17, 18, 20, 22],
 15: [3, 6, 9, 11, 12, 17, 19, 20],
 16: [2, 4, 5, 10, 14, 22],
 17: [1, 2, 4, 14, 15, 19, 20, 22],
 18: [0, 2, 4, 13, 14, 22],
 19: [15, 17, 20],
 20: [1, 2, 4, 14, 15, 17, 19, 22],
 21: [],
 22: [1, 2, 4, 10, 13, 14, 16, 17, 18, 20],
 23: []}
def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT#/2+abs(x.sum()-args.card)/args.card/2
def CB(alpha,x,M):
    return alpha*np.sqrt(np.dot(np.dot(x.T,np.linalg.inv(M)),x))
def solver(theta,MODEL):

    import gurobipy
    import time
    s=time.time()
    # 创建模型
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    # 更新变量环境
    MODEL.update()

    # 创建目标函数
    MODEL.setObjective(np.array(variables).dot(theta), sense=gurobipy.GRB.MAXIMIZE)

    # 创建约束条件
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    # 执行最优化
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_quad(Q,MODEL):

    import gurobipy
    import time
    s=time.time()
    # 创建模型
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    # 更新变量环境
    MODEL.update()

    # 创建目标函数
    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables)), sense=gurobipy.GRB.MAXIMIZE)

    # 创建约束条件
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    # 执行最优化
    
    print('??')
    MODEL.optimize()
    print('??')
    return np.array(MODEL.x)
a=10*(np.load('rateListUsersYahoo.npy')[0]-np.load('rateListUsersYahoo.npy')[0].min())#np.random.random(24) #  np.load('rateListUsersYahoo.npy')[0]
#10*(np.load('rateListml100.npy')-np.load('rateListml100.npy').min())
a /= np.linalg.norm(a, ord=2)

In [4]:
import numpy as np
import torch
import copy
eps=0.03
jitter=0.05
beta=0.2
rho=0.1
b=200
N=100
K=20
L=100000
constraintMean=0.2
def PPOobj(uNext,u):
    obj=0.
    for i in range(K):
        for j in range(args.solDim):
            if actions[i][j]==1:
                obj+=(uNext[j]/u[j])*(b-rewards[i])
            else:
                obj+=((1-uNext[j])/(1-u[j]))*(b-rewards[i])
            obj-=(beta*(u[j]*torch.log(u[j]/uNext[j])))
            obj-=(beta*(u[j]*torch.log(u[j]/uNext[j])))
    return obj
def feedback(x):
    return (a.dot(x))**3-10*constraintMean*checkFea(x)
actions=np.zeros([K,args.solDim])
rewards=np.zeros(K)
actionsN=np.zeros([N,args.solDim])
rewardsN=np.zeros(N)
constraintN=np.zeros(N)
actionhistoryBest=np.zeros([20,args.solDim])
actionhistoryWorst=np.zeros([20,args.solDim])
rewardhistoryBest=np.zeros(20)
rewardhistoryWorst=np.zeros(20)
u=np.random.random([args.solDim])
uNext=u.copy()
action=[np.random.choice([0,1],size=1,p=[1-u[i],u[i]])[0] for i in range(args.solDim)]
reward=sum(action)-20*checkFea(action)
actions[0]=action
rewards[0]=feedback(action)
actionsN[0]=action
rewardsN[0]=feedback(action)
actionhistoryBest[0]=action
actionhistoryWorst[0]=action
rewardhistoryBest[0]=feedback(action)
rewardhistoryWorst[0]=feedback(action)

for t in range(1,L):
    u=uNext.copy()
    action=[np.random.choice([0,1],size=1,p=[1-u[i],u[i]])[0] for i in range(args.solDim)]
    reward=sum(action)-20*checkFea(action)
    actions[(t%K)]=action
    rewards[(t%K)]=feedback(action)
    actionsN[(t%N)]=action
    rewardsN[(t%N)]=feedback(action)
    constraintN[(t%N)]=checkFea(action)
    if t<20:
        actionhistoryBest[t]=action
        actionhistoryWorst[t]=action
        rewardhistoryBest[t]=feedback(action)
        rewardhistoryWorst[t]=feedback(action)
    else:
        if reward>min(rewardhistoryBest) and checkFea(action)<0.15:
            actionhistoryBest[np.argwhere(rewardhistoryBest==min(rewardhistoryBest))]=action
            rewardhistoryBest[np.argwhere(rewardhistoryBest==min(rewardhistoryBest))]=feedback(action)
     #   if reward<max(rewardhistoryWorst) and feedback(action)<0.15:
      #      actionhistoryWorst[np.argwhere(rewardhistoryWorst==min(rewardhistoryWorst))]=action
       #     rewardhistoryWorst[np.argwhere(rewardhistoryWorst==min(rewardhistoryWorst))]=feedback(action)
    print(t,reward,checkFea(action))
    if t%K==0 and t%N!=0:
        b=rewards.mean()
        uNext=torch.tensor(uNext,requires_grad=True)
        optimizer=torch.optim.Adam([uNext],lr=1e-2)
        for step in range(5):
            pre=-PPOobj(uNext,u)
            print(step,pre)
            #pre.requires_grad=True
            optimizer.zero_grad()
            pre.backward()
            optimizer.step()
           # if step % 2000==0:
            #    print(step,uNext.tolist(),pre)      
        uNext=uNext.detach().numpy()
        uNext=np.clip(uNext,jitter,1-jitter)
    if t%N==0:
        cnt=0
        pointer=0
        elite=[]
        while cnt<int(N*rho) and pointer<N:
            if constraintN[np.argsort(rewardsN)[pointer]]<0.15:
                elite.append(np.argsort(rewardsN)[pointer])              
                cnt+=1
            pointer+=1
        if cnt<int(N*rho):
            while cnt<int(N*rho) and pointer<N:
                if np.argsort(rewardsN)[pointer] not in elite:
                    elite.append(np.argsort(rewardsN)[pointer])
                pointer+=1
        tmp=actionsN[elite].reshape([len(elite),args.solDim]).T
        for i in range(args.solDim):
            uNext[i]=max(min(np.exp(-t/L)*uNext[i] +(1-np.exp(-t/L))*np.sum(tmp[i])/len(elite),1-jitter),jitter)
    #    tmphistoryBest=actionhistoryBest.T
     #   for i in range(args.solDim):
      #      uNext[i]=max(min(uNext[i]/2+np.sum(tmphistoryBest[i])/len(elite)/2,1-jitter),jitter)
        constraintMean=constraintN.mean()

1 4.428571428571429 0.42857142857142855
2 5.8730158730158735 0.20634920634920634
3 8.603174603174605 0.2698412698412698
4 6.142857142857143 0.14285714285714285
5 6.238095238095238 0.23809523809523808
6 7.142857142857143 0.14285714285714285
7 7.682539682539683 0.015873015873015872
8 6.825396825396826 0.15873015873015872
9 5.380952380952381 0.38095238095238093
10 7.73015873015873 0.06349206349206349
11 6.968253968253968 0.30158730158730157
12 6.8730158730158735 0.20634920634920634
13 5.777777777777778 0.1111111111111111
14 6.238095238095238 0.23809523809523808
15 6.507936507936508 0.1746031746031746
16 8.650793650793652 0.31746031746031744
17 4.793650793650794 0.4603174603174603
18 4.333333333333334 0.3333333333333333
19 7.015873015873016 0.3492063492063492
20 7.142857142857143 0.14285714285714285
0 tensor(-1.3145e-12, dtype=torch.float64, grad_fn=<NegBackward>)
1 tensor(-16.1196, dtype=torch.float64, grad_fn=<NegBackward>)
2 tensor(-32.1870, dtype=torch.float64, grad_fn=<NegBackward>)
3

0 tensor(-5.0804e-13, dtype=torch.float64, grad_fn=<NegBackward>)
1 tensor(-5.2248, dtype=torch.float64, grad_fn=<NegBackward>)
2 tensor(-10.3775, dtype=torch.float64, grad_fn=<NegBackward>)
3 tensor(-15.4556, dtype=torch.float64, grad_fn=<NegBackward>)
4 tensor(-20.4546, dtype=torch.float64, grad_fn=<NegBackward>)
161 4.555555555555555 0.2222222222222222
162 6.412698412698413 0.07936507936507936
163 6.555555555555555 0.2222222222222222
164 6.095238095238095 0.09523809523809523
165 7.4603174603174605 0.12698412698412698
166 6.015873015873016 0.3492063492063492
167 5.095238095238095 0.09523809523809523
168 7.142857142857143 0.14285714285714285
169 7.412698412698413 0.07936507936507936
170 8.142857142857142 0.14285714285714285
171 5.333333333333334 0.3333333333333333
172 5.777777777777778 0.1111111111111111
173 5.920634920634921 0.25396825396825395
174 7.825396825396826 0.15873015873015872
175 6.555555555555555 0.2222222222222222
176 7.682539682539683 0.015873015873015872
177 4.412698412

KeyboardInterrupt: 

In [ ]:
elite

In [ ]:
全局最佳、sliding window、actionhistoryWorst（吸取教训

In [ ]:
def feedback(x):
    return (a.dot(x))**3-10*constraintMean*checkFea(x)
action=[np.random.choice([0,1],size=1,p=[1-u[i],u[i]])[0] for i in range(args.solDim)]
reward=sum(action)-20*checkFea(action)
actions[0]=action
rewards[0]=feedback(action)
actionsN[0]=action
rewardsN[0]=feedback(action)
actionhistoryBest[0]=action
rewardhistoryBest[0]=feedback(action)

for t in range(1,L):
    u=uNext.copy()
    action=[np.random.choice([0,1],size=1,p=[1-u[i],u[i]])[0] for i in range(args.solDim)]
    reward=sum(action)-20*checkFea(action)
    actions[(t%K)]=action
    rewards[(t%K)]=feedback(action)
    actionsN[(t%N)]=action
    rewardsN[(t%N)]=feedback(action)
    constraintN[(t%N)]=checkFea(action)
    if t<20:
        actionhistoryBest[t]=action
        rewardhistoryBest[t]=feedback(action)
    else:
        if reward>min(rewardhistoryBest) and checkFea(action)<0.15:
            actionhistoryBest[np.argwhere(rewardhistoryBest==min(rewardhistoryBest))]=action
            rewardhistoryBest[np.argwhere(rewardhistoryBest==min(rewardhistoryBest))]=feedback(action)
    print(t,reward,checkFea(action))
    if t%K==0 and t%N!=0:
        b=rewards.mean()
        uNext=torch.tensor(uNext,requires_grad=True)
        optimizer=torch.optim.Adam([uNext],lr=1e-2)
        for step in range(5):
            pre=-PPOobj(uNext,u)
            print(step,pre)
            #pre.requires_grad=True
            optimizer.zero_grad()
            pre.backward()
            optimizer.step()
           # if step % 2000==0:
            #    print(step,uNext.tolist(),pre)      
        uNext=uNext.detach().numpy()
        uNext=np.clip(uNext,jitter,1-jitter)
    if t%N==0:
        cnt=0
        pointer=0
        elite=[]
        while cnt<int(N*rho) and pointer<N:
            if constraintN[np.argsort(rewardsN)[pointer]]<0.15:
                elite.append(np.argsort(rewardsN)[pointer])              
                cnt+=1
            pointer+=1
        if cnt<int(N*rho):
            while cnt<int(N*rho) and pointer<N:
                if np.argsort(rewardsN)[pointer] not in elite:
                    elite.append(np.argsort(rewardsN)[pointer])
                pointer+=1
        tmp=actionsN[elite].reshape([len(elite),args.solDim]).T
        for i in range(args.solDim):
            uNext[i]=max(min(np.exp(-t/L)*uNext[i] +(1-np.exp(-t/L))*np.sum(tmp[i])/len(elite),1-jitter),jitter)
        constraintMean=constraintN.mean()